In [3]:
import numpy as np
import time
# 常量
k = 8.99e9  # 电场常数，单位 N·m²/C²
q = 1e-9    # 点电荷的电荷量，单位 C

# 定义空间范围和分辨率
space_size = 2.0  # 空间大小，单位 m
resolution = 0.01  # 分辨率，单位 m

# 生成空间网格
x = np.arange(-space_size/2, space_size/2+resolution, resolution)
nx = x.shape[0]
y = np.arange(-space_size/2, space_size/2+resolution, resolution)
ny = y.shape[0]
z = np.arange(-space_size/2, space_size/2+resolution, resolution)
nz = z.shape[0]
X, Y, Z = np.meshgrid(x, y, z)

# 计算电场强度
# 计算电场强度
R = np.sqrt((X**2 + Y**2 + Z**2))
theta = np.arctan2(Y, X)
E = np.zeros_like(X)
radius = 0.5
# 圆环上的电荷分布
for phi in np.linspace(0, 2*np.pi, 100):
    x_charge = radius * np.cos(phi)
    y_charge = radius * np.sin(phi)
    z_charge = 0
    R_charge = np.sqrt((X - x_charge)**2 + (Y - y_charge)**2 + (Z - z_charge)**2)
    E += k * q / R_charge**2

from scipy.interpolate import RegularGridInterpolator
interp_func = RegularGridInterpolator((x, y, z), E, bounds_error=False, fill_value=None)


# 处理电荷位置处的无穷大值
E[R == 0] = np.inf
from r8hermt import r8herm_spline,r8herm_interpolation
time1 = time.time()
E_spline = r8herm_spline(x, y, z, E)
time2 = time.time()
xget = 0.245
yget = 0.5
zget = 0.7672322
Eget = r8herm_interpolation(xget, yget, zget, x, y, z, E_spline)

time3 = time.time()
E_regular = interp_func(np.array([[xget], [yget], [zget]]).T)
time4 = time.time()


time_cost1 = time2-time1
time_cost2 = time3-time2
time_cost3 = time4-time3
# print(f"time to generate spline:{time_cost1:.3e} s")   #time to generate spline
print(f"time to get E_r8herm:{time_cost2:.3e} s")   #time to get E
print(f"time to get E_RegularGridInterpolator:{time_cost3:.3e} s")   #time to get E
print (f"r8herm_interpolation: {Eget:.5e}")
print(f"RegularGridInterpolator:{E_regular[0]:.5e}")

time to get E_r8herm:2.951e-03 s
time to get E_RegularGridInterpolator:1.187e-02 s
r8herm_interpolation: 8.99707e+02
RegularGridInterpolator:8.99735e+02


In [ ]:
%reset -f

# from scipy.integrate import solve_ivp
from r8hermt import r8herm_interpolation
import numpy as np
import time

temp_path = "/home/zkg/h1_scan/temp"
spline_data = np.load(f"{temp_path}/spline_data.npz")

bp_spline = spline_data['bp_spline']
# br_spline = spline_data['br_spline']
# bz_spline = spline_data['bz_spline']
r_grid = spline_data['r_grid']
z_grid = spline_data['z_grid']
phiarc_grid = spline_data['phiarc_grid']
phi_arcrange = phiarc_grid[:,0,0] # phi range in first dimesion
r_range = r_grid[0,:,0]
z_range = z_grid[0,0,:]
delta_phi = phiarc_grid[1,0,0] - phiarc_grid[0,0,0]
print("Spline data loaded")

phi_arcrange = phiarc_grid[:,0,0] # phi range in first dimesion
r_range = r_grid[0,:,0]
z_range = z_grid[0,0,:]
delta_phi = phiarc_grid[1,0,0] - phiarc_grid[0,0,0]
coil_currents = {
    'tf_1': 50, 'tf_2': 50, 'tf_3': 50, 'tf_4': 50,
    'tf_5': 50, 'tf_6': 50, 'pf': 5, 'hw': 5,
    'ovf': -80, 'ivf': -40
}

extcur = np.array(list(coil_currents.values())) 
bp_spline_extcur = bp_spline * extcur[:, np.newaxis, np.newaxis, np.newaxis, np.newaxis] 
bp_spline_extcur = np.sum(bp_spline_extcur, axis=0)
bp_spline_extcur = np.asfortranarray(bp_spline_extcur)
# br_spline_extcur = br_spline * extcur[:, np.newaxis, np.newaxis, np.newaxis, np.newaxis]
# br_spline_extcur = np.sum(br_spline_extcur, axis=0)
# bz_spline_extcur = bz_spline * extcur[:, np.newaxis, np.newaxis, np.newaxis, np.newaxis]
# bz_spline_extcur = np.sum(bz_spline_extcur, axis=0)

time_start = time.time()
bp_test = r8herm_interpolation(0, 1.25, 0, phi_arcrange, r_range, z_range, bp_spline_extcur)
time_end = time.time()
print(f"Interpolation done in {time_end - time_start:.3e} s")

import r8herm3
ilinx, iliny, ilinz, ier = 0, 0, 0, 0
ict = np.array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int)  # 插值控制参数
fval = np.zeros(1, dtype=float)  # 存储插值结果的数组
x = np.asfortranarray(phi_arcrange)  # 将 x_array 转换为 Fortran 风格数组
y = np.asfortranarray(r_range)  # 将 y_array 转换为 Fortran 风格数组
z = np.asfortranarray(z_range)  # 将 z_array 转换为 Fortran 风格数组
time_start2 = time.time()
bp_test2 = r8herm3.r8herm3ev(0, 1.25, 0, x, y, z, ilinx, iliny, ilinz, bp_spline_extcur, ict, fval, ier)
time_end2 = time.time()
print(f"Interpolation done in {time_end2 - time_start2:.3e} s")